In [1]:
from scipy.sparse import coo_matrix
import bioframe as bf
import bioframe.vis
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import gridspec
from matplotlib import cm
from matplotlib.gridspec import GridSpec
from matplotlib.gridspec import GridSpecFromSubplotSpec
import matplotlib.colors as colors
from matplotlib.colors import ListedColormap
%matplotlib inline

from collections import Counter
import itertools
import sys
from collections import OrderedDict
import random
import seaborn as sns
import scipy
import pickle


# post-processing: filtering

In [2]:
##nf-core atac pipeline was used in default mode to generate paired end bams.  
##Downstream analysis and QC were done using ful reads (centres)
##first I will call peaks using SE read ends of either full reads or reads <= 120bp for different analyses


In [3]:
#1. filter mapq30, dups, reduced chroms


In [4]:
# PCR dups,mapq30,keep paired
# bam='/mG_REP1.mLb.clN.sorted.bam'
# out='/mG_REP1.sorted.nodups.mapq30.t2.bam'
# !bsub -q short -W 08:00 -n 4 -R "span[hosts=1]" -R "rusage[mem=8000]" \
#         "samtools view -f 0x2 -q 30 -b -h -F 1024 $bam > $out"
    

In [5]:
#sort (if done in separate steps)

# bam='/mG_REP1.sorted.nodups.mapq30.t2.bam'
# out='/mG_REP1.sorted.nodups.mapq30.t2.sorted.bam'
# !bsub -q short -W 08:00 -n 4 -R "span[hosts=1]" -R "rusage[mem=8000]" \
#         "samtools sort $bam > $out"
    

In [6]:
#index
# bam='/mG_REP1.sorted.nodups.mapq30.t2.sorted.bam'
# !bsub -q short -W 08:00 -n 4 -R "span[hosts=1]" -R "rusage[mem=8000]" \
#         "samtools index $bam"
    

In [7]:
#keep only chroms (no M)

# bam='mG_REP1.sorted.nodups.mapq30.t2.sorted.bam'
# out='/mG_REP1.sorted.nodups.mapq30.t2.sorted.reducedchroms.bam'
# !bsub -q short -W 08:00 -n 4 -R "span[hosts=1]" -R "rusage[mem=8000]" \
#         "samtools view -h $bam chr1 chr2 chr3 chr4 chr5 chr6 chr7 chr8 chr9 chr10 chr11 chr12 chr13 chr14 chr15 chr16 chr17 chr18 chr19 chr20 chr21 chr22 > $out"


In [8]:
#sort again

# bam='/mG_REP1.sorted.nodups.mapq30.t2.sorted.reducedchroms.bam'
# out='/mG_REP1.nodups.mapq30.t2.sorted.reducedchroms.sorted.bam'
# !bsub -q short -W 08:00 -n 4 -R "span[hosts=1]" -R "rusage[mem=8000]" \
#         "samtools sort $bam > $out"


In [9]:
#index again

# bam='/mG_REP1.nodups.mapq30.t2.sorted.reducedchroms.sorted.bam'
# !bsub -q short -W 08:00 -n 4 -R "span[hosts=1]" -R "rusage[mem=8000]" \
#         "samtools index $bam"
    

# <120bp reads (can center for peak calling)


In [15]:
#if desired keep only < 120bp frags

# bam='/mG_REP1.nodups.mapq30.t2.sorted.reducedchroms.sorted.bam'
# out='/mG_REP1.nodups.mapq30.t2.sorted.reducedchroms.sorted.FL120.bam'
# !samtools view -h $bam | awk 'substr($0,1,1)=="@" || ($9>= 0 && $9<=120) || ($9<=0 && $9>=-120)' | samtools view -b > $out


In [16]:
#sort again

# bam='/mG_REP1.nodups.mapq30.t2.sorted.reducedchroms.sorted.FL120.bam'
# out='mG_REP1.nodups.mapq30.t2.sorted.reducedchroms.sorted.FL120.sorted.bam'
# !bsub -q short -W 08:00 -n 4 -R "span[hosts=1]" -R "rusage[mem=8000]" \
#         "samtools sort $bam > $out"
    

Job <156926> is submitted to queue <short>.


In [18]:
#index again

# bam='/mG_REP1.nodups.mapq30.t2.sorted.reducedchroms.sorted.FL120.sorted.bam'
# !bsub -q short -W 08:00 -n 4 -R "span[hosts=1]" -R "rusage[mem=8000]" \
#         "samtools index $bam"
    

Job <156974> is submitted to queue <short>.


# single end bed for peak calling: keep ends

In [10]:
#make single end bed. the samtoolsflags uneccessary given the above filtering steps

# bam='/pM_REP2.nodups.mapq30.t2.sorted.reducedchroms.sorted.bam'
# out='/pM_REP2.reducedchroms.sorted.bed'
# !bsub -q short -W 08:00 -n 4 -R "span[hosts=1]" -R "rusage[mem=8000]" \
#         "samtools view -bf 0x2 $bam | bedtools bamtobed -i stdin >| $out"


Job <447821> is submitted to queue <short>.


# shift reads for tn5

In [10]:

#!cat /Mp.reducedchroms.sorted.bed | awk -F $'\t' 'BEGIN {OFS = FS}{ if ($6 == "+") {$2= $2 + 4} else if ($6 == "-") {$3 = $3 - 5} print $0}' >| /Mp.reducedchroms.sorted.p4m5.bed

##for coverage, keep only end, so make 2nd step:
#!cat /Mp.reducedchroms.sorted.p4m5.bed | awk -F $'\t' 'BEGIN {OFS = FS}{ if ($6 == "+") {$2= $2-1;$3 = $2 + 2;} else if ($6 == "-") {$3 = $3 + 1;$2 = $3 - 2;} print $0}' >| Mp.reducedchroms.sorted.p4m5_ends.bed



# make pooled mitotic set

In [12]:
#sort -k1,1 -k2,2n mM_R1.reducedchroms.sorted.p4m5.bed pM_R1.reducedchroms.sorted.p4m5.bed > MR1.reducedchroms.sorted.p4m5.bed
#sort -k1,1 -k2,2n mM_R2.reducedchroms.sorted.p4m5.bed pM_R2.reducedchroms.sorted.p4m5.bed > MR2.reducedchroms.sorted.p4m5.bed
#sort -k1,1 -k2,2n mMp.reducedchroms.sorted.p4m5.sorted.bed pMp.reducedchroms.sorted.p4m5.sorted.bed > Mp.reducedchroms.sorted.p4m5.sorted.bed


# make bigwigs scaled to 10E6 reads


In [13]:
  # 191456639 pG_REP1.reducedchroms.sorted.p4m5.ends.bed 0.05223114775351301
  # 146370798 pG_REP2.reducedchroms.sorted.p4m5.ends.bed 0.06831963845684574
  # 200086671 pM_REP1.reducedchroms.sorted.p4m5.ends.bed 0.04997834163576043
  # 117654257 pM_REP2.reducedchroms.sorted.p4m5.ends.bed 0.0849947996356817
  # 184887917 mG_REP1.reducedchroms.sorted.p4m5.ends.bed 0.054086822774903134
  # 163581501 mG_REP2.reducedchroms.sorted.p4m5.ends.bed 0.06113160680681124
  # 147253247 mM_REP1.reducedchroms.sorted.p4m5.ends.bed 0.06791021728709316
  # 151592262 mM_REP2.reducedchroms.sorted.p4m5.ends.bed 0.06596642775869391


In [14]:
#genomecov bed

# bed='/mM_REP2.reducedchroms.sorted.p4m5.ends.bed'
# chroms='/hg38.reduced.chrom.sizes'
# out='/mM_REP2.genomcov.scaled.bed'
# !bsub -q short -W 04:00 -n 2 -R "span[hosts=1]" -R "rusage[mem=8000]" \
#         "bedtools genomecov -bg -scale 0.06596642775869391 -i $bed -g $chroms > $out" 


In [15]:
#sort genomecov bed

# bed1='/mM_REP2.genomcov.scaled.bed'
# out='/mM_REP2.genomcov.scaled.sorted.bed'
# !bsub -q short -W 04:00 -n 2 -R "span[hosts=1]" -R "rusage[mem=8000]" \
#         "sort -k1,1 -k2,2n $bed1 > $out"


In [16]:
#bedGraphToBigWig myFile_sorted.bedgraph myChrom.sizes myBigWig.bw



# call peaks c MACS3

In [10]:
#callpeak (narrow) shift and extend

# bed='/Mp.reducedchroms.sorted.p4m5.sorted.bed'
# outdir='/macs3peaks'
# !bsub -q short -W 08:00 -n 4 -R "span[hosts=1]" -R "rusage[mem=8000]" \
#         "macs3 callpeak -t $bed -g hs -f BED --nomodel --shift -75 --extsize 150 --name allM --outdir $outdir"



Job <335956> is submitted to queue <short>.


#   coverage at peaks


# peakscov.py

In [20]:
#peakscov.py

# import bioframe
# import pandas as pd
# import numpy as np
# import scipy
# import sys
# import os

# arguments=sys.argv

# directory=arguments[1]
# cov=arguments[2]
# peaks=arguments[3]
# outdir=arguments[4]
# logdir=arguments[5]

# f = open(f'{logdir}/{cov}.log','w')
# f.close()
# f=open(f'{logdir}/{cov}.log','a')
# f.write("go\n")
# f.flush()

# ##first cov
# #files
# cov_df=pd.read_csv(f'{directory}/{cov}/{cov}.genomcov.raw.bed',sep='\t',names=['chrom','start','end','cov'])
# peaks_df=pd.read_csv(f'{peaks}',sep='\t',names=['chrom','start','end','Mscore','mGscore','pGscore'])
# f.write("files in\n")
# f.flush()

# #overlap peaks with cov
# peaks_cov=bioframe.overlap(peaks_df,cov_df,how='left',suffixes=('','_cov'))
# f.write("overlap\n")
# f.flush()

# #calculate cov at peaks
# cov_mat = peaks_cov.groupby(['chrom','start','end'])['cov_cov'].sum()

# f.write("counts\n")
# f.flush()

# #save
# cov_mat_df = cov_mat.reset_index()
# cov_mat_df.to_csv(f'{outdir}/{cov}.out', sep='\t', index=False)

# f.write("fini\n")
# f.close()


In [21]:
#import os

# #1
# directory='/coverage/files'
# #3
# peaks='/union_R1R3_narrowPeaks_bothfirst_pmit.sorted.bed'
# #4
# outdir='/coverage/cov_allpeaks'
# #5
# logdir='/coverage/log'

# for cov in os.listdir(directory):
#     if os.path.isdir(os.path.join(directory,cov)):
#         !bsub -q short -W 04:00 -n 4 -R "span[hosts=1]" -R rusage[mem=24000] \
#             "source activate open2c2v2; python /coverage/peakscov.py {directory} {cov} {peaks} {outdir} {logdir}"
